DICOM is a standard file format used in medical imaaging.<br>
We need to extract pixel values and save them as jpg/png image in order to visualize and move forward with training.<br>
Along with pixel values, there is more data about patient in the file which might be useful for training.

The following code extracts png image and csv file from DICOM images.

In [ ]:
input_dir="/kaggle/input/siim-covid19-detection"

In [ ]:
%%file dicom_image_description.csv
Description,Code
SpecificCharacterSet,CS
SOPClassUID,UI
SOPInstanceUID,UI
StudyDate,DA
StudyTime,TM
AccessionNumber,SH
Modality,CS
PatientName,PN
PatientID,LO
PatientSex,CS
BodyPartExamined,CS
StudyInstanceUID,UI
SeriesInstanceUID,UI
StudyID,SH
SeriesNumber,IS
InstanceNumber,IS
SamplesperPixel,US
PhotometricInterpretation,CS
Rows,US
Columns,US
BitsAllocated,US
BitsStored,US
HighBit,US
PixelRepresentation,US
PixelData,OB

### offline install gdcm handler for pydicom

In [ ]:
!cp ../input/gdcm-conda-install/gdcm.tar .
!tar -xvzf gdcm.tar
!conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2
print("done")

### Extract Images

In [ ]:
import pydicom as dicom
import matplotlib.pyplot as plt
import os
import glob
import cv2
import pandas as pd
import csv

# Specify the .dcm folder path
images_path = glob.glob(input_dir+"/train/**/*.dcm", recursive=True)[:5]
# Specify dest folder path
dest_folder_path = "/kaggle/tmp"
# list of attributes available in dicom image
# download this file from the given link # https://github.com/vivek8981/DICOM-to-JPG
dicom_image_description = pd.read_csv("dicom_image_description.csv")

# make dirs
os.makedirs('/kaggle/tmp/', exist_ok=True)
os.makedirs('/kaggle/tmp/train_images/', exist_ok=True)

with open(dest_folder_path+'/train_dcom_details.csv', 'w', newline ='') as csvfile:
    fieldnames = list(dicom_image_description["Description"])
    writer = csv.writer(csvfile, delimiter=',')
    # write header
    writer.writerow(fieldnames)
    for n, image_path in enumerate(images_path):
        ds = dicom.dcmread(image_path)
        rows = []
        pixel_array_numpy = ds.pixel_array
        
        image = os.path.basename(image_path).replace('.dcm', '.png')
        cv2.imwrite(os.path.join(dest_folder_path, 'train_images', image), pixel_array_numpy)
        if n % 100 == 0:
            print('{} image converted'.format(n))
        for field in fieldnames:
            if ds.data_element(field) is None:
                rows.append('')
            else:
                x = str(ds.data_element(field)).replace("'", "")
                y = x.find(":")
                x = x[y+2:]
                rows.append(x)
        writer.writerow(rows)

In [ ]:
plt.imshow(cv2.imread('/kaggle/tmp/train_images/d54f6204b044.png',0), cmap='gray')

In [ ]:
pd.read_csv(dest_folder_path+'/train_dcom_details.csv').head()